In [1]:
from azureml.core import Workspace
from aml_obs.management import provision
import pandas as pd
from datetime import timedelta
from aml_obs.collector import Online_Collector

ws = Workspace.from_config()

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

### Provision Resources

#### Create ADX Cluster, Service Principal, and store value's in Azure Machine Learning's Key Vault for future use

In [ ]:
provision(ws)

### Data Collection

Once the resources are created, data can now be ingested to Azure Data Explorer. To use the dashboards, the data must have a timestamp column. 

In [7]:
# Get raw data

dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")

In [31]:
# Add timestamp column

dataset["datetime"] =  [pd.to_datetime('now') - timedelta(days=x) for x in range(len(dataset))]

C:\Users\nserafino\Anaconda3\envs\mlopsenv\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [32]:
# Save data

dataset.to_parquet("data/iris.parquet", index=False)

#### Batch Ingestion

Entire dataframe will be loaded into ADX at once as a table named ```irisdata```. The is also a stream ingestion available to ingest data asynchronously with an internal buffering mechanism. This method can be utilized to lower impact to main scoring thread.

In [33]:

table_name = "irisdata" #new dataset

sample_pd_data = pd.read_parquet("data/iris.parquet").head(10)

online_collector = Online_Collector(table_name,ws=ws)
online_collector.batch_collect(sample_pd_data)

In [27]:
# get cluster uri to use in browser

online_collector.cluster_uri

'https://adx02.westus2.kusto.windows.net'